<a href="https://colab.research.google.com/github/Mikko812/Pub/blob/master/dohod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tkinter import *
import tkinter as tk
import tkinter.ttk as ttk
from datetime import datetime as dt
from datetime import timedelta as td
import pandas as pd
import numpy as np
import json
import time

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
www = 'https://www.dohod.ru/ik/analytics/dividend/'
MY_ACT={'Газпром':2000, 'НОВАТЭК': 50, 'ВТБ': 250, 'Роснефть': 200, 'Сургутнефтегаз-п': 1000, 'Сбербанк-ао': 700, 'ММК': 1500, 'Юнипро': 30000,
   'Россети Московский регион (МОЭСК)': 45000, 'НЛМК': 200}
MY_OFZ = {'26240': 150, '26243': 150, '29006': 79, '29009': 100, '29010': 6, '26207': 400, '26229': 260, '26238': 220, '26244': 90, '29014': 150}
FLOATERS = {'29015': 0, '29022': 0, '29019': 0, '29008': 1.4, '29009': 1.5, '29010': 1.6, '29013': 0, 'ТФинанс1P1': 2.75}
MY_OFZ_ACC = {'26240': 'О', '26243': 'О', '29006': 'И, О', '29009': 'О', '29010': 'И, О', '26207': 'И', '29014': 'И', '26229': 'И', '26238': 'И', '26244': 'И'}
MY_OBL = {'РСХБ2Р2': 150, 'НорНикБ1P7': 80, 'Газпн3P10R': 100, 'РоссЦP03': 190, 'ВТБ Б1-351': 120, 'МВ ФИНАНС 001Р-03': 130, 'ТФинанс1P1': 80,
          'РЖД 1Р-26R': 75, 'ИКС5Фин3P2': 50, 'ПСБ 3P-09': 70}
MY_OBL_CODES = {'РоссЦP03': 'RU000A107AG6', 'ВТБ Б1-351': 'RU000A107W55', 'Газпн3P10R': 'RU000A107UW1', 'НорНикБ1P7': 'RU000A1083A6', 'РСХБ2Р2': 'RU000A1068R1',
                'ТФинанс1P1': 'RU000A109KT4', 'МВ ФИНАНС 001Р-03': 'RU000A104ZK2', 'РЖД 1Р-26R': 'RU000A106K43', 'ИКС5Фин3P2': 'RU000A1075S4', 'ПСБ 3P-09': 'RU000A109TJ6' }

!pip install free-proxy
from fp.fp import FreeProxy
def get_proxy():
  proxy = FreeProxy(rand=True).get()   # , https=True
  proxies = {'http': proxy}
  print(f'Proxy: {proxy}')
  return proxies

In [4]:
r = requests.get('https://fin-plan.org/bonds/rus/', headers=headers)
s = BeautifulSoup(r.text, 'html.parser')
div = s.find_all(class_='linkList')

print(len(div))

2393


In [17]:
# Все облигации с доходом от 5% до 33%
rows = []
for d in div:
    code = d.find('a').get('href').split('/')[-2]
    info = d.get_text().split(' (')
    name = info[0]
    pro = float(info[1].split('%,\xa0')[0])
    date = info[1].split('%,\xa0')[1][:-2]
    if 35 > pro > 5:
      rows.append([name, code, pro, date])


df = pd.DataFrame(np.array(rows, dtype=object), columns='name code pro date'.split())
df = df.sort_values('pro', ascending=False).reset_index(drop=True)
df

,name,code,pro,date
0,АПРИ 2Р3,RU000A106WZ2,34.99,14.09.2027
1,ГлобалФ1P2,RU000A1060U2,34.99,08.09.2026
2,МВ ФИН 1Р3,RU000A104ZK2,34.88,22.07.2025
3,ФЭСАгро1Р1,RU000A106P63,34.78,06.08.2026
4,СберИОС416,RU000A103976,34.72,05.07.2028
5,ТАЛК001P01,RU000A108E98,34.63,04.05.2026
6,ЭконЛиз1Р4,RU000A1067T9,34.63,12.05.2026
7,РоялКапБ10,RU000A1079H9,34.6,24.11.2026
8,ТЛК 1Р1,RU000A109RF8,34.51,23.09.2027
9,Рубеж 1P2,RU000A107YT9,34.48,06.03.2029


In [25]:
df.query("name.str.startswith('ПСБ')")

,name,code,pro,date
736,ПСБ 3P-09,RU000A109TJ6,21.04,15.10.2027
1705,ПСБ 02,RU000A0JXJM3,6.48,
1706,ПСБ обл03,RU000A0JXJP6,6.47,


In [20]:
df_ofz = df[df.name.str.match('ОФЗ')].reset_index(drop=True)
df_ofz

,name,code,pro,date
0,ОФЗ 26236,SU26236RMFS8,22.44,17.05.2028
1,ОФЗ 26232,SU26232RMFS7,21.9,06.10.2027
2,ОФЗ 26235,SU26235RMFS0,21.5,12.03.2031
3,ОФЗ 26219,SU26219RMFS4,21.3,16.09.2026
4,ОФЗ 26212,SU26212RMFS9,21.24,19.01.2028
5,ОФЗ 26237,SU26237RMFS6,21.23,14.03.2029
6,ОФЗ 26226,SU26226RMFS9,21.19,07.10.2026
7,ОФЗ 26207,SU26207RMFS9,21.07,03.02.2027
8,ОФЗ 26229,SU26229RMFS3,20.91,12.11.2025
9,ОФЗ 26239,SU26239RMFS2,20.76,23.07.2031


In [ ]:
rows = []
for code in df_ofz['code']:
  print(code)
  proxies = get_proxy()
  r2 = requests.get(f"https://www.dohod.ru/assets/components/dohodbonds/connectorweb.php?action=getbondinfo&isin={code}", headers=headers, proxies=proxies).json()
  rows.append([code, r2['frnIndex'], r2['frnIndexAdd'], r2['creditRatingText'], r2['addInfo']['nkd'], r2['price'],  r2['yearsToMaturity'], r2['priceReturn'], r2['couponSize']])
  #time.sleep(3)

df3 = pd.DataFrame(np.array(rows, dtype=object), columns='code index add rating nkd price years doxod coupon'.split())
df_ofz = df_ofz.merge(df3, on='code', how='left')
df_ofz

RU000A1014N4
Proxy: http://128.140.113.110:1080
RU000A0ZZYW2
Proxy: http://13.56.192.187:80
RU000A1028E3
Proxy: http://64.92.82.58:8080
RU000A0JWM07
Proxy: http://13.208.56.180:80
RU000A1038Z7
Proxy: http://44.219.175.186:80
RU000A0JS3W6
Proxy: http://15.236.106.236:3128
RU000A102BT8
Proxy: http://18.223.25.15:80
RU000A100EG3
Proxy: http://13.208.56.180:80
RU000A0JTK38
Proxy: http://3.136.29.104:80
RU000A101QE0
Proxy: http://43.201.121.81:80
RU000A0ZYUA9
Proxy: http://43.201.121.81:80
RU000A103901
Proxy: http://54.67.125.45:3128
RU000A100A82
Proxy: http://44.195.247.145:80
RU000A1066D5
Proxy: http://35.76.62.196:80
RU000A105RV3
Proxy: http://204.236.176.61:3128
RU000A105G16
Proxy: http://13.56.192.187:80
RU000A0JVW48
Proxy: http://204.236.176.61:3128
RU000A101F94
Proxy: http://13.56.192.187:80
RU000A0GN9A7
Proxy: http://204.236.137.68:80
RU000A0JXFM1
Proxy: http://18.228.198.164:80
RU000A103BR0
Proxy: http://8.219.97.248:80
RU000A100EF5
Proxy: http://54.67.125.45:3128
RU000A0ZYUB7
Prox

,name,code,pro,date,index,add,rating,nkd,price,years,doxod,coupon
0,ОФЗ 26232,RU000A1014N4,21.62,06.10.2027,,,AAA,6.08,72.081,2.89,19.88,29.92
1,ОФЗ 26226,RU000A0ZZYW2,21.15,07.10.2026,,,AAA,8.06,81.899,1.9,20.85,39.64
2,ОФЗ 26235,RU000A1028E3,21.07,12.03.2031,,,AAA,9.38,58.382,6.33,17.55,29.42
3,ОФЗ 26219,RU000A0JWM07,21.06,16.09.2026,,,AAA,12.31,82.086,1.84,20.82,38.64
4,ОФЗ 26237,RU000A1038Z7,21.04,14.03.2029,,,AAA,10.65,67.119,4.33,18.74,33.41
5,ОФЗ 26207,RU000A0JS3W6,20.84,03.02.2027,,,AAA,20.32,80.112,2.22,20.56,40.64
6,ОФЗ 26236,RU000A102BT8,20.77,17.05.2028,,,AAA,27.64,68.297,3.51,19.21,28.42
7,ОФЗ 26229,RU000A100EG3,20.77,12.11.2025,,,AAA,0.39,88.8,0.99,21.18,35.65
8,ОФЗ 26212,RU000A0JTK38,20.65,19.01.2028,,,AAA,22.02,72.999,3.18,19.55,35.15
9,ОФЗ 26234,RU000A101QE0,20.19,16.07.2025,,,AAA,14.92,90.54,0.67,21.4,22.44


In [ ]:
df_ofz.sort_values('nkd')

,name,code,pro,date,index,add,rating,nkd,price,years,doxod,coupon
24,ОФЗ 29022,RU000A105G16,19.43,20.07.2033,RUONIA,,AAA,3.16,95.021,8.71,0.0,None
21,ОФЗ 29024,RU000A1066D5,20.25,18.04.2035,RUONIA,,AAA,3.16,92.38,10.46,0.0,None
52,ОФЗ 52004,RU000A103MX5,8.8,17.03.2032,INFLATION_ROSSTAT,,AAA,3.71,58.292,7.37,11.17,None
4,ОФЗ 26228,RU000A100A82,22.01,10.04.2030,,,AAA,4.19,64.269,5.43,19.14,38.15
1,ОФЗ 26232,RU000A1014N4,22.53,06.10.2027,,,AAA,4.44,70.836,2.92,20.54,29.92
29,ОФЗ 46023,RU000A0JRTL6,18.51,23.07.2026,,,AAA,4.61,91.5,1.72,19.51,8.14
8,ОФЗ 26226,RU000A0ZZYW2,21.63,07.10.2026,,,AAA,5.88,81.215,1.92,21.24,39.64
28,ОФЗ 29015,RU000A1025A7,18.54,18.10.2028,RUONIA,,AAA,6.75,97.7,3.96,0.0,None
26,ОФЗ 29019,RU000A102A49,18.79,18.07.2029,RUONIA,,AAA,6.75,97.919,4.7,0.0,None
15,ОФЗ 26221,RU000A0JXFM1,21.1,23.03.2033,,,AAA,7.17,58.75,8.39,17.88,38.39


In [27]:
mobl = df.query(f"code in {list(MY_OBL_CODES.values())}")
mobl

,name,code,pro,date
2,МВ ФИН 1Р3,RU000A104ZK2,34.88,22.07.2025
583,РоссЦP03,RU000A107AG6,22.49,18.05.2027
673,ТФинанс1P1,RU000A109KT4,21.63,10.09.2026
681,Газпн3P10R,RU000A107UW1,21.58,12.02.2027
685,РЖД 1Р-26R,RU000A106K43,21.54,19.07.2028
700,НорНикБ1P7,RU000A1083A6,21.4,28.02.2029
736,ПСБ 3P-09,RU000A109TJ6,21.04,15.10.2027
764,РСХБ2Р2,RU000A1068R1,20.8,08.05.2026
804,ИКС5Фин3P2,RU000A1075S4,20.5,17.10.2026
813,ВТБ Б1-351,RU000A107W55,20.41,02.03.2027


In [28]:
rows = []
for code in mobl['code']:
  print(code)
  proxies = get_proxy()
  r1 = requests.get(f"https://www.dohod.ru/assets/components/dohodbonds/connectorweb.php?action=getbondinfo&isin={code}", headers=headers, proxies=proxies).json()
  rows.append([code, r1['frnIndex'], r1['frnIndexAdd'], r1['creditRatingText'], r1['addInfo']['nkd'], r1['price'],  r1['yearsToMaturity'], r1['priceReturn'], r1['couponSize']])
  #time.sleep(3)

df2 = pd.DataFrame(np.array(rows, dtype=object), columns='code index add rating nkd price years doxod coupon'.split())
df2

RU000A106WZ2
Proxy: http://8.219.97.248:80
RU000A1060U2
Proxy: http://8.219.97.248:80
RU000A104ZK2
Proxy: http://3.78.92.159:80
RU000A106P63
Proxy: http://52.67.10.183:80
RU000A103976
Proxy: http://52.67.10.183:80
RU000A108E98
Proxy: http://13.38.153.36:80
RU000A1067T9
Proxy: http://43.200.77.128:3128
RU000A1079H9
Proxy: http://13.37.59.99:80
RU000A109RF8
Proxy: http://13.38.153.36:80
RU000A107YT9
Proxy: http://13.36.113.81:3128
RU000A108FG3
Proxy: http://209.121.164.51:31147


,code,index,add,rating,nkd,price,years,doxod,coupon
0,RU000A106WZ2,,,B+,45.37,88.2,0.82,48.92,59.84
1,RU000A1060U2,,,CCC,3.18,86.82,1.79,39.94,11.93
2,RU000A104ZK2,,,BB,11.5,87.57,0.66,38.96,30.79
3,RU000A106P63,,,B+,6.9,78.5,1.7,35.97,34.9
4,RU000A103976,,,AA,0.0,None,3.62,0.0,0.7
5,RU000A108E98,,,B+,7.5,83.37,1.45,37.65,15.0
6,RU000A1067T9,,,B+,4.99,79.67,1.47,35.98,34.9
7,RU000A1079H9,,,CCC,49.32,77.06,2.01,37.76,49.86
8,RU000A109RF8,,,,12.08,96.6,0.86,33.23,20.14
9,RU000A107YT9,,,B,38.52,70.0,4.29,35.41,46.12


In [24]:
mobl.merge(df2, on='code', how='left')

,name,code,pro,date,index,add,rating,nkd,price,years,doxod,coupon
0,МВ ФИН 1Р3,RU000A104ZK2,34.88,22.07.2025,,,BB,11.5,87.57,0.66,38.96,30.79
1,РоссЦP03,RU000A107AG6,22.49,18.05.2027,CBR_RATE,0.9,AA,14.18,95.26,2.48,9492.71,16.36
2,ТФинанс1P1,RU000A109KT4,21.63,10.09.2026,CBR_RATE,2.75,BB,3.9,99.89,1.8,0.0,None
3,Газпн3P10R,RU000A107UW1,21.58,12.02.2027,CBR_RATE,1.3,AA,0.61,99.07,2.22,0.0,17.84
4,РЖД 1Р-26R,RU000A106K43,21.54,19.07.2028,RUONIA,1.3,AA,0.58,97.2,3.66,74.38,16.79
5,НорНикБ1P7,RU000A1083A6,21.4,28.02.2029,CBR_RATE,1.3,AA,2.44,97.32,4.27,82.43,18.33
6,РСХБ2Р2,RU000A1068R1,20.8,08.05.2026,RUONIA,1.5,A,15.88,98.6,1.46,1433.37,16.98
7,ИКС5Фин3P2,RU000A1075S4,20.5,17.10.2026,CBR_RATE,1.1,AA,15.97,98.89,1.9,41.86,16.52
8,ВТБ Б1-351,RU000A107W55,20.41,02.03.2027,CBR_RATE,0.25,AA,13.97,100.31,2.27,23.24,17.47


In [ ]:
# r1 = requests.get('https://www.dohod.ru/assets/components/dohodbonds/connectorweb.php?action=getbondinfo&isin=RU000A1038V6', headers=headers).json()

(r1['frnIndex'], r1['frnIndexAdd'], r1['creditRatingText'], r1['addInfo']['nkd'], r1['price'],  r1['yearsToMaturity'], r1['priceReturn'], r1['couponSize'])

('', '', 'AAA', 29.76, 48.744, 16.54, 16.63, 35.4)

In [ ]:
# ОФЗ
def full_sum(r):
    res = (int((r['Погашение'] - r['Дата купона']).days / 182) + 1) * r['Сумма купона']
    return res


r = requests.get('https://smart-lab.ru/q/ofz/', headers=headers)
s = BeautifulSoup(r.text, 'html.parser')
with open('ofz.html', 'w', encoding='cp1251') as f:
    f.write(str(s))
ofz = pd.read_html('ofz.html', encoding='cp1251')[0]
ofz = ofz.drop(['№', 'Unnamed: 2', 'Unnamed: 11', 'Unnamed: 17', 'Unnamed: 18'], axis=1)
ofz = ofz.rename(columns={'!': 'Тип'})
ofz = ofz.query("Тип != 'АД'")
ofz = ofz.sort_values(['Тип', 'Имя'])
ofz = ofz[ofz['Имя'].apply(lambda x: not x.startswith('ОФЗ 52') and not x.startswith('ОФЗ 25'))]
ofz.insert(0, 'Мои', '')
ofz['Мои'] = ofz['Имя'].apply(lambda x: MY_OFZ_ACC.get(x[-5:], 'x'))
ofz.insert(1, 'Кол-во', '')
ofz['Кол-во'] = ofz['Имя'].apply(lambda x: MY_OFZ.get(x[-5:], 0))
ofz['Погашение'] = ofz['Погашение'].apply(lambda x: dt.strptime(x, '%d.%m.%Y'))
ofz['Погашение'] = pd.to_datetime(ofz['Погашение'], format='%Y-%m-%d')
ofz['Дата купона'] = ofz['Дата купона'].apply(lambda x: dt.strptime(x, '%d.%m.%Y'))
ofz['Дата купона'] = pd.to_datetime(ofz['Дата купона'], format='%Y-%m-%d')
ofz['Доходн'] = ofz['Доходн'].apply(lambda x: float(x.replace('%', '')))
ofz['Цена'] = ofz['Цена'].apply(lambda x: float(x) if x != '-' else 0)
ofz['Объем,млн руб'] = ofz['Объем,млн руб'].apply(lambda x: float(x.replace(' ', '')) if x != '-' else 0)
ofz['Купон, руб'] = ofz['Купон, руб'].apply(lambda x: float(x) if x != '-' else 0)
ofz['НКД, руб'] = ofz['НКД, руб'].apply(lambda x: float(x))
ofz['Год.куп.дох.'] = ofz['Год.куп.дох.'].apply(lambda x: float(x.replace('%', '')))
ofz['Куп.дох.посл.'] = ofz['Куп.дох.посл.'].apply(lambda x: float(x.replace('%', '')) if x != '-' else 0)
ofz.insert(2, 'Сумма купона', '')
ofz['Сумма купона'] = (np.array(ofz['Кол-во']) * np.array(ofz['Купон, руб'])).astype('int')
ofz.insert(3, 'Общая сумма', '')
ofz['Общая сумма'] = ofz.apply(full_sum, axis=1)
cols = ofz.columns
#ofz

In [ ]:
def fill_tree():
    global ofz_tuple
    tree.tag_configure('my', background='#FFFF22')
    c = 0
    for c, v in enumerate(ofz_tuple):
        tree.insert(parent='', index='end', text='', iid=c, values=v, tags=('my',))
        #c += 1

def sort_df():
    global ofz_tuple, ofz
    for row in tree.get_children():
        tree.delete(row)
    cols_to_sort = []
    bool_to_sort = []
    c = 0
    for k, v in checkboxes.items():
        if v['var'].get():
            cols_to_sort.append(ofz.columns[c])
            bool_to_sort.append(bool_sort[c])
        c += 1
    tmp = ofz.sort_values(cols_to_sort, ascending=bool_to_sort)
    tmp['Погашение'] = tmp['Погашение'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))
    tmp['Дата купона'] = tmp['Дата купона'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))
    curr_col.config(text=' | '.join(cols_to_sort))
    ofz_tuple = tmp.itertuples(index=False, name=None)
    fill_tree()

def sort_df_by_zag(col):
    global ofz_tuple, ofz
    curr_col.config(text=col)
    for row in tree.get_children():
        tree.delete(row)
    if col in ['Мои', 'Кол-во', 'Доходн', 'Год.куп.дох.', 'Куп.дох.посл.', 'Сумма купона', 'Купон, руб']:
        tmp = ofz.sort_values(col, ascending=False)
    else:
        tmp = ofz.sort_values(col)
    ofz_tuple = tmp.itertuples(index=False, name=None)
    fill_tree()


bool_sort = [False, False, False, False, True, True, True, False, False, False, False, False, False, False, False, False, False, True]
c_width = [40, 60, 90, 90, 70, 80, 90, 60, 30, 80, 90, 50, 100, 80, 100, 60, 80, 90]
padxx =   [18, 10, 38, 32, 21, 27, 35, 20, 0, 27, 22, 22, 29,  27, 29, 23, 22, 22]
root = tk.Tk()
style = ttk.Style()
style.theme_use('default')
style.configure('Treeview.Heading', font=('Calibri', 10, 'bold'), background='#C8F')

root.geometry(f"{sum(c_width)}x750+200+50")
boxes_frame = ttk.Frame(root)
boxes_frame.grid(row=0, column=0)
checkboxes = {}
for i in range(len(c_width)):
    checked = BooleanVar()
    box = ttk.Checkbutton(boxes_frame, text=i+1, variable=checked, command=sort_df)
    box.grid(row=0, column=i, pady=6, padx=padxx[i])
    checkboxes[str(i)] = {'var': checked, 'box': box}

tree = ttk.Treeview(root, height=27)
tree.grid(row=1, column=0)
scrollbar = ttk.Scrollbar(orient='vertical', command=tree.yview)
tree['yscrollcommand'] = scrollbar.set
tree.tag_configure('my', background='#FF2')

buttons_frame = ttk.Frame(root, borderwidth=1, relief=SOLID, padding=[3, 3])
buttons_frame.grid(row=2, column=0, pady=11)
Label(buttons_frame, text='Текущая колонка:').grid(row=0, column=0)
curr_col = Label(buttons_frame)
curr_col.grid(row=0, column=1, columnspan=3)
root.columnconfigure(0, weight=1)

tree['columns'] = cols
tree.column('#0', width=0, stretch=NO)
for c, h in enumerate(cols):
    tree.column(f'#{c+1}', anchor=CENTER, width=c_width[c])
    tree.heading(f'#{c+1}', text=cols[c], anchor=CENTER)  #, command=lambda c=c: sort_df(cols[c]))
ofz_tuple = ofz.itertuples(index=False, name=None)

my_ofz = ofz.query("Мои != 'x'").sort_values('Дата купона').reset_index(drop=True)
Label(buttons_frame, text='Даты / Суммы купонов:').grid(row=1, column=0)
for c in range(len(my_ofz)):
    if c < 4:
        Label(buttons_frame, text=f"({my_ofz.iat[c, 4]}) - {dt.strftime(my_ofz.iat[c, 17], '%Y-%m-%d')}: {my_ofz.iat[c, 2]}.  ").grid(row=1, column=c % 4 + 1)
    else:
        Label(buttons_frame, text=f"({my_ofz.iat[c, 4]}) - {dt.strftime(my_ofz.iat[c, 17], '%Y-%m-%d')}: {my_ofz.iat[c, 2]}.  ").grid(row=2, column=c % 4 + 1)
fill_tree()


root.mainloop()

In [ ]:
my_ofz

,Мои,Кол-во,Сумма купона,Общая сумма,Имя,Погашение,Лет допогаш.,Доходн,Тип,Год.куп.дох.,Куп.дох.посл.,Цена,"Объем,млн руб","Купон, руб","Частота,раз в год","НКД, руб","Дюр-я, лет",Дата купона
0,True,282,9982,9982,ОФЗ 26222,2024-10-16,0.1,16.9,-,7.1,7.2,99.00,1 179.3,35.40,2,27.4,0.11,2024-10-16
1,True,250,8912,26736,ОФЗ 26229,2025-11-12,1.2,17.9,-,7.1,8.0,89.58,85.5,35.65,2,22.1,1.13,2024-11-13
2,True,100,8497,135952,ОФЗ 29009,2032-05-05,7.7,16.7,ПК,17.0,16.4,104.10,1.3,84.97,2,52.8,4.41,2024-11-13
3,True,220,7788,264792,ОФЗ 26238,2041-05-15,16.8,15.5,-,7.1,13.6,52.16,1 727.4,35.40,2,17.9,7.23,2024-12-04
4,True,150,7330,205240,ОФЗ 26243,2038-05-19,13.8,16.0,-,9.8,14.4,68.29,943.7,48.87,2,24.7,6.37,2024-12-04
5,True,6,517,10857,ОФЗ 29010,2034-12-06,10.3,17.2,ПК,17.3,16.6,103.97,11.2,86.31,2,37.0,5.07,2024-12-18
6,True,79,6692,6692,ОФЗ 29006,2025-01-29,0.4,18.0,ПК,17.0,17.0,99.85,104.3,84.72,2,16.8,0.40,2025-01-29
7,True,282,11460,57300,ОФЗ 26207,2027-02-03,2.4,17.7,-,8.2,9.8,83.11,66.2,40.64,2,6.5,2.21,2025-02-05
8,True,150,5235,125640,ОФЗ 26240,2036-07-30,11.9,15.9,-,7.0,12.7,54.93,116.6,34.90,2,4.2,6.80,2025-02-12


In [ ]:
def get_obls(sector_id='0'):
    global obl, obl_tuple
    page = 1
    tmp = [''] * 100
    while len(tmp) == 100:
        next_page = f"order_by_val_to_day/desc/page{page}/"
        if sector_id == '0':
            link = f"https://smart-lab.ru/q/bonds/{next_page}?rating_gt=A&rating_lt=AAA&bonds_amortized=0&bonds_variable=1&paids_year=12" #&mat_years_gt=0&mat_years_lt=3"   # -1 - все облигации, 1 - только флоатеры
        else:
            link = f"https://smart-lab.ru/q/bonds/{next_page}?sector_id%5B%5D={sector_id}&rating_gt=A&rating_lt=AAA&bonds_amortized=0&bonds_variable=1"
        r = requests.get(link, headers=headers)
        s = BeautifulSoup(r.text, 'html.parser')
        with open('oblig.html', 'w', encoding='cp1251') as f:
            f.write(str(s))
        tmp = pd.read_html('oblig.html', encoding='cp1251')[0].fillna(0)
        if page == 1:
            obl = tmp.copy()
        else:
            obl = pd.concat([obl, tmp])
        page += 1

    link = 'https://smart-lab.ru/q/bonds/order_by_short_name/asc/?paids_year=12&bonds_amortized=0&bonds_variable=1'
    r = requests.get(link, headers=headers)
    s = BeautifulSoup(r.text, 'html.parser')
    with open('oblig.html', 'w', encoding='cp1251') as f:
        f.write(str(s))
    tmp = pd.read_html('oblig.html', encoding='cp1251')[0].fillna(0)
    tmp = tmp.query("Имя == 'ТФинанс1P1'")
    obl = pd.concat([obl, tmp])


    obl = obl.drop(['№', 'Unnamed: 2', 'Unnamed: 18', 'Unnamed: 19', 'Оферта', 'Размещение'], axis=1)
    obl.columns = list(map(lambda x: x.replace('Частота,раз в год', 'Частота'), obl.columns))
    obl = obl[obl['Цена'].apply(lambda x: x > 0)]
    obl['Доходн'] = obl['Доходн'].apply(lambda x: float(x.replace('%', '').replace(' ', '')))
    obl['Год.куп.дох.'] = obl['Год.куп.дох.'].apply(lambda x: float(x.replace('%', '')))
    obl['Куп.дох.посл.'] = obl['Куп.дох.посл.'].apply(lambda x: float(x.replace('%', '')) if x != '-' else 0)
    obl.insert(9, 'Купон(год)', '')
    obl['Купон(год)'] = round(obl['Купон, руб'] * obl['Частота'], 2)
    obl['Цена'] = obl['Цена'].apply(lambda x: float(x) if x != '-' else 0)
    obl['Погашение'] = obl['Погашение'].apply(lambda x: dt.strptime(x, '%d.%m.%y'))
    obl['Погашение'] = pd.to_datetime(obl['Погашение'], format='%Y-%m-%d')
    obl['Дата купона'] = obl['Дата купона'].apply(lambda x: dt.strptime(x, '%d.%m.%y'))
    obl['Дата купона'] = pd.to_datetime(obl['Дата купона'], format='%Y-%m-%d')
    obl.insert(0, 'Мои', '')
    obl['Мои'] = obl['Имя'].apply(lambda x: MY_OBL.get(x, 0))
    obl.insert(11, 'Доход(период)', '')
    obl['Доход(период)'] = round(obl['Мои'] * obl['Купон, руб'], 2)
    obl = obl.sort_values(['Дата купона'])
    return obl

obl = get_obls()


In [ ]:



def fill_tree(obl_tuple):
    #global obl_tuple
    tree.tag_configure('my', background='#FFFF22')
    c = 0
    for c, v in enumerate(obl_tuple):
        tree.insert(parent='', index='end', text='', iid=c, values=v, tags=('my',))
        #c += 1

def sort_df():
    global obl_tuple, obl
    for row in tree.get_children():
        tree.delete(row)
    cols_to_sort = []
    bool_to_sort = []
    c = 0
    for k, v in checkboxes.items():
        if v['var'].get():
            cols_to_sort.append(obl.columns[c])
            bool_to_sort.append(bool_sort[c])
        c += 1
    tmp = obl.sort_values(cols_to_sort, ascending=bool_to_sort)
    tmp['Погашение'] = tmp['Погашение'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))
    tmp['Дата купона'] = tmp['Дата купона'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))
    curr_col.config(text=' | '.join(cols_to_sort))
    obl_tuple = tmp.itertuples(index=False, name=None)
    fill_tree(obl_tuple)


sectors = {'all': '0', 'machine': '11', 'gasoil': '1', 'banki': '2', 'electro': '12'}
cols = ['Мои', 'Имя', 'Лет допогаш.', 'Доходн', 'Год.куп.дох.', 'Куп.дох.посл.',
       'Рейтинг', 'Объем', 'Купон, руб', 'Частота', 'Купон(год)', 'Доход(период)',
       'НКД', 'Дюр-я', 'Цена', 'Дата купона', 'Погашение']
bool_sort = [False, True, True, False, False, False, True, False, False, False, False, False, False, False, False, True, True]
c_width = [50, 90, 90, 60, 90, 90, 60, 55, 75, 60, 80, 100, 40, 50, 50, 80, 80]
padxx =   [22, 25, 40, 10, 40, 25, 30, 0, 40, 8, 30, 25, 20, 5, 13, 15, 22]

root = tk.Tk()
style = ttk.Style()
style.theme_use('default')
style.configure('Treeview.Heading', font=('Calibri', 10, 'bold'), background='#C8F')

root.geometry(f"{sum(c_width)}x750+250+50")
boxes_frame = ttk.Frame(root)
boxes_frame.grid(row=0, column=0)
checkboxes = {}
for i in range(len(c_width)):
    checked = BooleanVar()
    box = ttk.Checkbutton(boxes_frame, text=i+1, variable=checked, command=sort_df)
    box.grid(row=0, column=i, pady=6, padx=padxx[i])
    checkboxes[str(i)] = {'var': checked, 'box': box}

tree = ttk.Treeview(root, height=27)
tree.grid(column=0, row=1)

buttons_frame = ttk.Frame(root, borderwidth=1, relief=SOLID, padding=[3, 3])
buttons_frame.grid(row=2, column=0, pady=11)
Label(buttons_frame, text='Тек. колонка:').grid(row=0, column=0)
curr_col = Label(buttons_frame)
curr_col.grid(row=0, column=1)

for c, sec in enumerate(sectors.keys()):
    Button(buttons_frame, text=sec, command=lambda sec=sec: get_obls(sectors[sec])).grid(row=0, column=c+2, padx=5, pady=11)
root.columnconfigure(0, weight=1)

scrollbar = ttk.Scrollbar(orient='vertical', command=tree.yview)
tree['yscrollcommand'] = scrollbar.set
tree.tag_configure('my', background='#FF2')

tree['columns'] = cols
tree.column('#0', width=0, stretch=NO)
for c, h in enumerate(cols):
    tree.column(f'#{c+1}', anchor=CENTER, width=c_width[c])
    tree.heading(f'#{c+1}', text=cols[c], anchor=CENTER)

if len(obl):
    my_obl = obl.query('Мои > 0').reset_index(drop=True)
    dates = pd.to_datetime(my_obl['Дата купона'], format='%d.%m.%y', exact=True)
    dates = [x.date() for x in dates]
    my_obl.loc[:, 'Дата купона'] = dates
    my_obl = my_obl.sort_values('Дата купона').reset_index(drop=True)
    Label(buttons_frame, text='Даты / Суммы купонов:').grid(row=2, column=0)
    for c in range(len(my_obl)):
        Label(buttons_frame, text=f"{my_obl.at[c, 'Имя']} - {my_obl.at[c, 'Дата купона']}: {my_obl.at[c, 'Доход(период)']}.  ").grid(row=2 + c // 3, column=c % 3 + 1)

obl_tuple = obl.itertuples(index=False, name=None)
fill_tree(obl_tuple)

root.mainloop()

In [ ]:
obl

,Мои,Имя,Лет допогаш.,Доходн,Год.куп.дох.,Куп.дох.посл.,Рейтинг,"Объем, млн руб","Купон, руб",Частота,Купон(год),Доход(период),"НКД, руб","Дюр-я, лет",Цена,Дата купона,Погашение
13,0,iПозитивР1,2.7,0.0,20.5,20.3,AA,25.9,16.88,12,202.56,0.0,-,-,101.00,2024-10-17,2027-07-04
55,0,Газпнф3P6R,3.5,23.7,17.3,17.4,AAA,1.7,43.13,4,172.52,0.0,42.7,0.24,99.62,2024-10-18,2028-04-14
12,0,ЕврХол3P01,2.1,0.0,20.1,20.1,AA+,26.7,16.52,12,198.24,0.0,16.0,-,100.15,2024-10-18,2026-11-07
10,0,ФосАгро П2,1.9,0.0,20.0,20.0,AAA,33.9,16.41,12,196.92,0.0,15.9,-,100.01,2024-10-18,2026-09-08
2,0,НоваБевБП7,2.8,20.0,19.4,19.4,AA,114.5,15.95,12,191.40,0.0,14.9,0.09,100.17,2024-10-19,2027-08-05
45,0,МОЭ 1Р01,4.8,0.0,20.3,20.2,AA+,4.2,16.68,12,200.16,0.0,15.6,-,100.50,2024-10-19,2029-07-25
83,0,ВТБ Б1-359,2.9,27.3,19.0,19.0,AAA,0.0,15.62,12,187.44,0.0,14.6,0.01,99.97,2024-10-19,2027-09-20
51,0,ГПБ005P02P,2.7,0.0,19.0,19.5,AA+,2.3,15.62,12,187.44,0.0,14.1,-,97.41,2024-10-20,2027-06-20
58,0,Магнит4P06,1.7,-40.4,19.0,18.9,AAA,0.8,15.62,12,187.44,0.0,14.1,0.01,100.59,2024-10-20,2026-07-12
64,0,РЖД 1Р-27R,2.9,-58.6,19.4,19.2,AAA,0.4,16.47,12,197.64,0.0,14.9,0.01,100.90,2024-10-20,2027-09-09


In [ ]:
my_obl = obl.query('Мои > 0').reset_index(drop=True)
dates = pd.to_datetime(my_obl['Дата купона'], format='%d.%m.%y', exact=True)
dates = [x.date() for x in dates]
my_obl.loc[:, 'Дата купона'] = dates
my_obl = my_obl.sort_values('Дата купона').reset_index(drop=True)
my_obl

,Мои,Имя,Лет допогаш.,Доходн,Год.куп.дох.,Куп.дох.посл.,Рейтинг,"Объем, млн руб","Купон, руб","Частота,раз в год",Купон(год),Доход(период),"НКД, руб","Дюр-я, лет",Цена,Дата купона,Погашение
0,80,РоссЦP03,2.7,57.7,16.9,17.0,AAA,7.8,13.89,12,166.68,1111.2,12.0,0.01,99.68,2024-08-31,18.05.27
1,120,ВТБ Б1-351,2.5,19.8,18.3,18.2,AAA,11.2,15.50,12,186.00,1860.0,13.0,2.01,100.04,2024-09-01,02.03.27
2,80,НорНикБ1P7,4.5,7.1,19.3,19.1,AAA,111.9,15.86,12,190.32,1268.8,2.1,0.07,100.88,2024-09-22,28.02.29
3,80,РСХБ2Р2,1.7,10.5,18.4,18.3,AA,35.1,15.61,12,187.32,1248.8,0.5,0.08,100.68,2024-09-26,08.05.26
4,80,Газпнф3P6R,3.6,19.1,17.3,17.3,AAA,3.4,43.13,4,172.52,3450.4,18.5,0.14,99.90,2024-10-18,14.04.28


In [ ]:
# КОРПОРАТИВНЫЕ ОБЛИГАЦИИ
sectors = {'machine': '11', 'gasoil': '1', 'banki': '2', 'electro': '12', }
link = f"https://smart-lab.ru/q/bonds/?sector_id%5B%5D={sectors['banki']}&rating_gt=AA&rating_lt=AAA&bonds_amortized=0&bonds_variable=1"
r = requests.get(link, headers=headers)
s = BeautifulSoup(r.text, 'html.parser')
with open('oblig.html', 'w', encoding='cp1251') as f:
    f.write(str(s))
obl = pd.read_html('oblig.html', encoding='cp1251')[0]
obl = obl.drop(['№', 'Unnamed: 2', 'Unnamed: 18', 'Unnamed: 19', 'Оферта'], axis=1)
obl = obl[obl['Купон, руб'].apply(lambda x: x > 0)]
obl = obl.sort_values(['Дата купона'])
cols = obl.columns
obl

,Имя,Лет допогаш.,Доходн,Год.куп.дох.,Куп.дох.посл.,Рейтинг,"Объем, млн руб","Купон, руб","Частота,раз в год","НКД, руб","Дюр-я, лет",Цена,Дата купона,Размещение,Погашение
2,ВТБ Б1-343,2.0,20.1%,18.0%,18.1%,AAA,11.6,15.29,12,0.5,1.70,99.29,01.09.24,01.09.23,01.09.26
3,ВТБ Б1-351,2.5,19.9%,18.3%,18.3%,AAA,11.0,15.50,12,0.5,2.03,99.98,01.09.24,01.03.24,02.03.27
13,ВТБ Б1-346,2.3,20.0%,18.0%,18.1%,AAA,3.7,15.29,12,0.5,1.87,99.32,01.09.24,01.12.23,01.12.26
20,ГПБ005P12P,3.4,0.0%,18.5%,18.5%,AA+,0.2,15.71,12,0.5,-,99.99,01.09.24,01.07.24,01.02.28
27,ПСБ БО-01,1.1,0.0%,16.2%,-,AA+,NaN,40.39,4,23.1,-,NaN,11.10.24,23.10.15,10.10.25
6,РСХБ2Р3,2.4,6.5%,17.6%,17.5%,AA,8.9,14.95,12,9.6,0.03,100.34,13.09.24,09.02.24,29.01.27
28,ПСБ БО-02,1.1,0.0%,16.2%,-,AA+,NaN,40.39,4,21.8,-,NaN,14.10.24,26.10.15,13.10.25
5,РСЭКСМБ2Р4,9.2,19.8%,20.0%,19.9%,AA,9.8,49.86,4,9.9,0.20,100.28,14.11.24,16.11.23,03.11.33
22,ДОМ.РФ15об,4.1,-76.8%,18.5%,17.4%,AAA,0.0,93.26,2,86.7,0.04,106.50,15.09.24,27.08.09,15.09.28
29,ПСБ БО-03,1.1,0.0%,16.2%,-,AA+,NaN,40.39,4,21.3,-,NaN,15.10.24,27.10.15,14.10.25


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
#Чтение и обработка таблицы
act = pd.read_html('https://www.dohod.ru/ik/analytics/dividend')[0]
act.drop(labels=['Unnamed: 4','Unnamed: 5','Unnamed: 9','Unnamed: 21','Unnamed: 22',
                 'Unnamed: 19','Unnamed: 17', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 18',
                 'Unnamed: 24', 'Капитализация, sorting', 'Капитализация, млн. руб.', 'DSI'], inplace=True, axis=1)
act.rename(columns={'Unnamed: 12':'Прогноз прибыли','Unnamed: 13':'Доля от прибыли','Unnamed: 14':'Кол-во акций',
                    'Капитализация, млн. USD':'Капит., млн. USD', 'Цена расчета доходности':'Цена расчета дох-ти',
                    'Unnamed: 20':'Nik', 'Доходность выплаты':'Дох-ть выплаты',
                    'Выплата на акцию (прогноз)': 'Выплата на акцию', 'Дата закрытия реестра (оценка)': 'Дата закрытия реестра'}, inplace=True)
act.dropna(axis=0, inplace=True)
#dd = act['Дата закрытия реестра']
#act['Дата закрытия реестра'] = list(map(lambda x: f'{x[6:]}.{x[3:5]}.{x[:2]}',dd))
dates = pd.to_datetime(act['Дата закрытия реестра'], format='%d.%m.%Y', exact=True)
dates = [x.date() for x in dates]
act.loc[:, 'Дата закрытия реестра'] = dates
company = list(set(act['Акция']))

tmp = [x.replace('%','') for x in act['Дох-ть выплаты'].to_list()]
act['Дох-ть выплаты'] = [float(x) for x in tmp]
act.insert(1, 'Мои', '')
act['Мои'] = act['Акция'].apply(lambda x: MY_ACT.get(x, 0))
act = act.reset_index(drop=True)
act.insert(2, 'Дивы', '')
act['Дивы'] = round(act['Мои'] * act['Выплата на акцию'], 2)
display(act.sort_values(by=['Дата закрытия реестра'], ascending=True).head())

,Акция,Мои,Дивы,Сектор,Период,Выплата на акцию,Дох-ть выплаты,Цена расчета дох-ти,Дата закрытия реестра,Прогноз прибыли,Доля от прибыли,Кол-во акций,Nik
209,Полюс,0,0.0,Металлы и добыча,I полугодие 2024,0.0,0.00,13003.00,2024-10-07,143348.00,0.00,136.07,plzl
47,Татнефть-п,0,0.0,Нефть/Газ,I полугодие 2024,38.2,6.21,614.80,2024-10-08,151016.00,3.73,147.51,tatnp
48,Татнефть-ао,0,0.0,Нефть/Газ,I полугодие 2024,38.2,6.18,618.00,2024-10-08,151016.00,55.11,2178.69,tatn
41,Селигдар ао,0,0.0,Металлы и добыча,1 полугодие 2024,4.0,7.06,56.69,2024-10-11,323.24,1274.60,1030.00,selg
81,НОВАТЭК,30,1065.0,Нефть/Газ,I полугодие 2024,35.5,3.61,982.20,2024-10-11,330958.62,32.57,3036.31,nvtk


In [ ]:
def fill_tree():
    global act_tuple
    tree.tag_configure('my', background='#FFFF22')
    c = 0
    for v in act_tuple:
        tree.insert(parent='', index='end', text='', iid=c, values=v, tags=('my',))
        c += 1

def sort_df(col):
    global act_tuple, act
    curr_col.config(text=col)
    for row in tree.get_children():
        tree.delete(row)
    if col in ['Мои', 'Кол-во', 'Выплата на акцию', 'Дох-ть выплаты', 'Дата закрытия реестра',
               'Сумма купона', 'Купон, руб', 'Дивы', 'Дата закрытия реестра']:
        act = act.sort_values(col, ascending=False)
    else:
        act = act.sort_values(col)
    act = act.reset_index(drop=True)
    act_tuple = act.itertuples(index=False, name=None)
    fill_tree()

def item_selected(event):
    global act
    idx = int(tree.selection()[0])
    stock = act.at[idx, 'Акция']
    text_frame.delete('1.0', END)

    curr = act.query(f"'{stock}' in Акция")
    nik = curr.loc[curr.index[0],'Nik']
    r = requests.get(f'{www}{nik}',headers=headers)
    s = BeautifulSoup(r.text,'html.parser')
    c = s.find('div',id='rightside-col')
    try:
        com = c.get_text().split('Комментарий:\n')[1][:-1]
        text_frame.insert('1.0', com)
    except AttributeError:
        text_frame.insert('1.0', 'НЕТ КОММЕНТАРИЯ.')
    except IndexError:
        text_frame.insert('1.0', 'НЕТ КОММЕНТАРИЯ.')


c_width = [200, 60, 60, 120, 100, 110, 100, 120, 130, 110, 110, 90, 50]
root = tk.Tk()
style = ttk.Style()
style.theme_use('default')
style.configure('Treeview.Heading', font=('Calibri', 10, 'bold'), background='#C8F')
cols = act.columns

root.geometry(f"{sum(c_width)}x750+200+50")
tree = ttk.Treeview(root, height=27)
tree.grid(column=0, row=0)
scrollbar = ttk.Scrollbar(orient='vertical', command=tree.yview)
tree['yscrollcommand'] = scrollbar.set
tree.tag_configure('my', background='#FF2')
tree.bind("<<TreeviewSelect>>", item_selected)

text_frame = tk.Text(root, width=150, height=7, wrap='word')
text_frame.grid(row=2, column=0, pady=11)

buttons_frame = ttk.Frame(root, borderwidth=1, relief=SOLID, padding=[3, 3])
buttons_frame.grid(row=1, column=0, pady=11)
Label(buttons_frame, text='Текущая колонка:').grid(row=0, column=0)
curr_col = Label(buttons_frame)
curr_col.grid(row=0, column=1)
root.columnconfigure(0, weight=1)

tree['columns'] = cols
tree.column('#0', width=0, stretch=NO)
tree.column('#1', anchor=W, width=c_width[0])
tree.heading('#1', text=cols[0], anchor=W, command=lambda c=0: sort_df(cols[c]))
for c, h in enumerate(cols[1:]):
    tree.column(f'#{c+2}', anchor=CENTER, width=c_width[c+1])
    tree.heading(f'#{c+2}', text=cols[c+1], anchor=CENTER, command=lambda c=c: sort_df(cols[c+1]))
act_tuple = act.itertuples(index=False, name=None)

'''my_ofz = ofz.query('Мои > 0').sort_values('Дата купона').reset_index(drop=True)
Label(buttons_frame, text='Даты / Суммы купонов:').grid(row=1, column=0)
for c in range(len(my_ofz)):
    if c < 4:
        Label(buttons_frame, text=f"({my_ofz.iat[c, 4]}) - {my_ofz.iat[c, 17]}: {my_ofz.iat[c, 2]}.  ").grid(row=1, column=c % 4 + 1)
    else:
        Label(buttons_frame, text=f"({my_ofz.iat[c, 4]}) - {my_ofz.iat[c, 17]}: {my_ofz.iat[c, 2]}.  ").grid(row=2, column=c % 4 + 1) '''
fill_tree()


root.mainloop()

In [ ]:
# Мой портфель
def calc(r):
    r['MyDoxod'] = r['Doxod'] * MY_ACT[r['Asset']]
    return r


assets = act[['Акция', 'Период',  'Дата закрытия реестра', 'Выплата на акцию']]
assets.columns = ['Asset', 'Period', 'Date', 'Doxod']
k = list(MY_ACT.keys())
my_assets = assets.loc[assets.Asset.isin(k)]
my_assets.insert(4, 'MyDoxod', 0)
my_assets = my_assets.apply(calc, axis=1)
display(round(my_assets['MyDoxod'].sum(), 2), my_assets.sort_values('Date'))

103173.9

,Asset,Period,Date,Doxod,MyDoxod
157,Газпром,I полугодие 2024,2024-10-11,0.0000,0.0
81,НОВАТЭК,I полугодие 2024,2024-10-11,35.5000,1065.0
198,НЛМК,II квартал 2024,2024-10-12,0.0000,0.0
56,ММК,II квартал 2024,2024-10-17,2.4900,3735.0
192,Юнипро,9 месяцев 2024,2024-12-17,0.0000,0.0
50,Роснефть,9 месяцев 2024,2025-01-11,30.5200,6104.0
54,НЛМК,III квартал 2024,2025-01-11,7.9000,1580.0
72,ММК,III квартал 2024,2025-01-13,1.9700,2955.0
61,НОВАТЭК,II полугодие 2024,2025-03-26,50.6200,1518.6
174,Юнипро,годовые 2024,2025-05-13,0.0000,0.0


In [ ]:
def clicked():
    select=combobox.get()
    print(select)
    if select == 'Дата закрытия реестра (оценка)':
        display(tab.sort_values(by=[select, 'Дох-ть выплаты'], ascending=[True, False]))
    else:
        display(tab.sort_values([select], ascending=False))
    root.destroy()

root = tk.Tk()
values = ['Период', 'Выплата на акцию (прогноз)', 'Дох-ть выплаты', 'Цена расчета дох-ти', 'Дата закрытия реестра (оценка)']
select='-'
root.geometry('400x50+700+150')
frame = tk.Frame(root)
btn=Button(root,text="Выбрать",command=clicked)
btn.grid(column=2, row=0)
frame.grid(column=1,row=0)
combobox = ttk.Combobox(frame, values=values, height=22)
combobox.grid(column=1,row=0)#Позиционируем Combobox на форме

root.mainloop()

Выплата на акцию (прогноз)


,Акция,Сектор,Период,Выплата на акцию (прогноз),Дох-ть выплаты,Цена расчета дох-ти,Дата закрытия реестра (оценка),"Капитализация, млн. руб.",DSI,Прогноз прибыли в след. 12m,Доля от прибыли,Кол-во акций,Стаб. выплат,Стаб. роста,Unnamed: 18,Nik,"Капитализация, sorting",Unnamed: 24
94,"АКБ ""Приморье""",Финансы и Банки,II полугодие 2024,2000.000000,6.29,31800.000000,2025.05.06,7 950.00,0.50,1685.43,32.50,0.25,1.00,0.00,500.00,prmb,0,7950
45,ЛУКОЙЛ,Нефть/Газ,годовые 2024,677.420000,10.48,6463.500000,2025.05.07,4 478 337.85,1.00,647225.09,72.52,692.87,1.00,1.00,469359.00,lkoh,0,4478338
34,Магнит,Розничная торговля,годовые 2024,659.400000,11.87,5557.000000,2025.07.15,566 321.40,0.29,67356.80,99.77,101.91,0.29,0.00,67200.00,mgnt,0,566321
179,ВСМПО-АВИСМА,Металлы и добыча,годовые 2024,560.480000,1.97,28400.000000,2025.06.03,327 438.88,0.00,18306.11,35.30,11.53,0.00,0.00,6462.06,vsmo,0,327439
80,ЛУКОЙЛ,Нефть/Газ,9 месяцев 2024,458.470000,7.09,6463.500000,2024.12.17,4 478 337.85,1.00,647225.09,49.08,692.87,1.00,1.00,317656.80,lkoh,0,4478338
151,Полюс,Металлы и добыча,I полугодие 2024,401.310000,3.24,12396.000000,2024.10.07,1 686 716.28,0.00,109534.00,49.85,136.07,0.00,0.00,54605.96,plzl,0,1686716
97,Магнит,Розничная торговля,9 месяцев 2024,330.470000,5.95,5557.000000,2025.01.11,566 321.40,0.29,33678.40,100.00,101.91,0.29,0.00,33678.40,mgnt,0,566321
88,НоваБев Групп (Белуга Групп),Продовольствие,I полугодие 2024,325.900000,6.51,5007.000000,2024.09.29,79 110.60,0.57,5149.28,100.00,15.80,0.71,0.43,5149.28,belu,0,79111
164,Саратовский НПЗ-ап,Нефть/Газ,годовые 2024,264.730000,2.67,9920.000000,2025.07.08,2 473.42,0.29,660.08,10.00,0.25,0.29,0.00,66.01,krknp,0,2473
72,Башнефть-ао,Нефть/Газ,годовые 2024,217.930000,7.97,2734.500000,2025.07.12,450 219.34,0.29,148013.75,21.26,144.42,0.43,0.14,31472.35,bane,0,450219


In [ ]:
def clicked():
    select=combobox.get()
    print(select)
    try:
        curr=tab.query(f"'{select}' in Акция")
        nik=curr.loc[curr.index[0],'Nik']
        r=requests.get(f'{www}{nik}',headers=headers)
        s=BeautifulSoup(r.text,'html.parser')
        c=s.find('div',id='rightside-col')
        com=c.get_text().split('Комментарий:\n')[1][:-1]
        print(com)
        display(curr)
        print()
    except AttributeError:
        print('НЕТ КОММЕНТАРИЯ.')
        print()

root = tk.Tk()

select='-'
root.geometry('200x50+700+150')
frame = tk.Frame(root)
btn=Button(root,text="Выбрать",command=clicked)
btn.grid(column=2, row=0)
frame.grid(column=1,row=0)
combobox = ttk.Combobox(frame,values=sorted(company),height=22)
combobox.grid(column=1,row=0)#Позиционируем Combobox на форме

root.mainloop()

Whoosh
Дивидендная политика Whoosh привязана к чистой прибыли и показателю «чистый долг/EBITDA». Если он ниже 1,5, акционерам направляется 50% чистой прибыли, если в диапазоне 1,5–2,5 – 25% чистой прибыли, если выше 2,5 – дивиденды не выплачиваются.


,Акция,Сектор,Период,Выплата на акцию (прогноз),Дох-ть выплаты,Цена расчета дох-ти,Дата закрытия реестра (оценка),"Капит., млн. USD",DSI,Прогноз прибыли в след. 12m,Доля от прибыли,Кол-во акций,Стаб. выплат,Стаб. роста,Unnamed: 18,Nik,"Капитализация, sorting",Unnamed: 24
156,Whoosh,Разное,9 месяцев 2024,7.0,2.38,294.48,2024.12.04,32 799.90,0.07,1559.32,50.0,111.38,0.14,0.0,779.66,wush,0,32800



АФК Система
Согласно дивидендной политике (март 2024 г.) на 2024-2026 гг. АФК «Система» стремится выплачивать один раз в год дивиденды в размере не менее 0,52 руб. на 1 обыкновенную акцию в 2024 г., далее в 2025-2026 гг. планируется ежегодно увеличивать размер дивидендов на 25-50% относительно предыдущей выплаты. Кроме того, при существенном сокращении внешнего долга компании Совет директоров может рекомендовать направлять на выплату дивидендов до 50% от величины чистого денежного потока за отчетный год. Под чистым денежным потоком понимается сумма поступлений от монетизации инвестиций, а также дивидендов, выплачиваемых в пользу компании, уменьшенная на инвестиционные вложения, расходы на привлечение финансирования, кадровые и административно-управленческие расходы.


,Акция,Сектор,Период,Выплата на акцию (прогноз),Дох-ть выплаты,Цена расчета дох-ти,Дата закрытия реестра (оценка),"Капит., млн. USD",DSI,Прогноз прибыли в след. 12m,Доля от прибыли,Кол-во акций,Стаб. выплат,Стаб. роста,Unnamed: 18,Nik,"Капитализация, sorting",Unnamed: 24
164,АФК Система,Разное,годовые 2023,0.52,2.2,23.65,2024.07.17,228 193.55,0.2,0.0,0.0,9650.0,0.14,0.0,5018.0,afks,1,228194



ГК Самолет
Согласно дивидендной политики ГК Самолет (октябрь 2020 г.) определенны следующие минимальные уровни выплат: при соотношении чистый долг/скорр. EBITDA меньше 1,0x на выплату дивидендов будет направлено не менее 50% чистой прибыли предыдущего финансового периода по МСФО; при показателе чистый долг/скорректированная EBITDA от 1 до 2 - не менее 33%. При этом при любой долговой нагрузке сумма дивидендов составит не менее 5 млрд руб. в год. Возможна замена дивидендов на выкуп акций.


,Акция,Сектор,Период,Выплата на акцию (прогноз),Дох-ть выплаты,Цена расчета дох-ти,Дата закрытия реестра (оценка),"Капит., млн. USD",DSI,Прогноз прибыли в след. 12m,Доля от прибыли,Кол-во акций,Стаб. выплат,Стаб. роста,Unnamed: 18,Nik,"Капитализация, sorting",Unnamed: 24
182,ГК Самолет,Строительство,I полугодие 2024,40.6,1.35,3017.0,2024.10.12,185 784.92,0.0,12185.47,20.52,61.58,0.0,0.0,2500.0,smlt,0,185785
183,ГК Самолет,Строительство,9 месяцев 2024,40.6,1.35,3017.0,2024.12.10,185 784.92,0.0,18278.20,13.68,61.58,0.0,0.0,2500.0,smlt,0,185785
224,ГК Самолет,Строительство,годовые 2023,0.0,0.00,3017.0,2024.07.10,185 784.92,0.0,16364.00,0.00,61.58,0.0,0.0,0.0,smlt,0,185785
284,ГК Самолет,Строительство,I квартал 2024,0.0,0.00,3017.0,2024.07.11,185 784.92,0.0,6092.73,0.00,61.58,0.0,0.0,0.0,smlt,0,185785


In [ ]:
curr=tab.query("'Газпром' in Акция")
nik=curr.loc[curr.index[0],'Nik']
r=requests.get(f'{www}{nik}',headers=headers)
s=BeautifulSoup(r.text,'html.parser')
c=s.find('div',id='rightside-col')
com=c.get_text().split('Комментарий:\n')[1][:-1]
com

'Согласно дивидендной политике, отчисления на выплаты дивидендов составляют 10% от прибыли по РСБУ. При достижении целевого уровня резервов - от 17,5% до 35% прибыли. Ранее платили 25% от чистой прибыли по РСБУ. По итогам 2018г рекомендовали 27% от прибыли по МСФО. По итогам 2019г обещают на дивиденды направить 30% от прибыли по МСФО, скорректированной на неденежные статьи, по итогам 2020г - 40%, по итогам 2021г выйдут на целевой уровень в 50%.'

In [ ]:
root = Tk()

def callback(var):
       print ("%d" %var)

menubar = Menu(root)

# create a pulldown menu, and add it to the menu bar
filemenu = Menu(menubar, tearoff=0)
filemenu.add("command", label="Open", command=lambda: callback(1))
filemenu.add("command", label="Save", command=lambda: callback(2))
filemenu.add_separator()
filemenu.add_command(label="Exit", command=root.quit)
menubar.add_cascade(label="File", menu=filemenu)

# create more pulldown menus
editmenu = Menu(menubar, tearoff=0)
editmenu.add("command", label="Cut", command=lambda: callback(3))
editmenu.add("command", label="Copy", command=lambda: callback(4))
editmenu.add("command", label="Paste", command=lambda: callback(5))
menubar.add_cascade(label="Edit", menu=editmenu)

helpmenu = Menu(menubar, tearoff=0)
helpmenu.add("command", label="About", command=lambda: callback(6))
menubar.add_cascade(label="Help", menu=helpmenu)

# display the menu
root.config(menu=menubar)

root.mainloop()